In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle


In [7]:
df = pd.read_csv("exercise_angles.csv")
features = [
    "Shoulder_Angle", "Elbow_Angle", "Hip_Angle", "Knee_Angle", "Ankle_Angle",
    "Shoulder_Ground_Angle", "Elbow_Ground_Angle", "Hip_Ground_Angle", "Knee_Ground_Angle", "Ankle_Ground_Angle"
]

sequence_length = 120
X_seq, y_seq = [], []

# Create sequences
for i in range(0, len(df) - sequence_length + 1):
    window = df.iloc[i:i + sequence_length]
    if window['Label'].nunique() == 1:
        X_seq.append(window[features].values)
        y_seq.append(window['Label'].iloc[0])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_seq)

In [8]:
class ExerciseSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [9]:
class CNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super(CNN_LSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=5, padding=2)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(input_size=64, hidden_size=hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch, features, seq_len)
        x = self.relu(self.conv1(x))
        x = x.permute(0, 2, 1)  # (batch, seq_len, channels)
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

In [10]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=20):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs = model(X_val)
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == y_val).sum().item()
                total += y_val.size(0)
        print(f"Validation Accuracy: {correct / total:.4f}")

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_encoded, test_size=0.2)
train_dataset = ExerciseSequenceDataset(X_train, y_train)
val_dataset = ExerciseSequenceDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Initialize model and train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN_LSTM(input_size=10, hidden_size=64, num_layers=2, num_classes=len(label_encoder.classes_))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
train_model(model, train_loader, val_loader, criterion, optimizer, device)

# Save model and config
torch.save(model.state_dict(), "cnn_lstm_model.pth")
with open("model_config.pkl", "wb") as f:
    pickle.dump({
        "input_size": 10,  # ← was 5 before
        "hidden_size": 64,
        "num_layers": 2,
        "num_classes": len(label_encoder.classes_),
        "dropout": 0.3
    }, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

Epoch 1, Loss: 0.5469
Validation Accuracy: 0.8860
Epoch 2, Loss: 0.1903
Validation Accuracy: 0.9768
Epoch 3, Loss: 0.0929
Validation Accuracy: 0.9627
Epoch 4, Loss: 0.0975
Validation Accuracy: 0.9630
Epoch 5, Loss: 0.0541
Validation Accuracy: 0.9974
Epoch 6, Loss: 0.0241
Validation Accuracy: 0.9975
Epoch 7, Loss: 0.0385
Validation Accuracy: 0.9644
Epoch 8, Loss: 0.0701
Validation Accuracy: 0.9970
Epoch 9, Loss: 0.0328
Validation Accuracy: 0.9936
Epoch 10, Loss: 0.0378
Validation Accuracy: 0.9979
Epoch 11, Loss: 0.0168
Validation Accuracy: 0.9980
Epoch 12, Loss: 0.0309
Validation Accuracy: 0.9404
Epoch 13, Loss: 0.0385
Validation Accuracy: 0.9957
Epoch 14, Loss: 0.0312
Validation Accuracy: 0.9997
Epoch 15, Loss: 0.0137
Validation Accuracy: 0.9951
Epoch 16, Loss: 0.0052
Validation Accuracy: 0.9990
Epoch 17, Loss: 0.0137
Validation Accuracy: 0.9987
Epoch 18, Loss: 0.0165
Validation Accuracy: 0.9997
Epoch 19, Loss: 0.0189
Validation Accuracy: 0.9997
Epoch 20, Loss: 0.0026
Validation Accura